In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque
import random
from env import Env
import numpy as np
import wandb
import copy

# wandb setup
number = 2
NAME = "DeepSARSA_HER" + str(number)
ID = "Deep SARSA_HER" + str(number)
run = wandb.init(project='DeepSARSA_HER_MachineReplacement', name = NAME, id = ID)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'reward', 'next_state', 'next_action'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DeepSARSA(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DeepSARSA, self).__init__()
        self.layer1 = nn.Linear(n_observations, 16)
        self.layer2 = nn.Linear(16, 16)
        self.layer3 = nn.Linear(16, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [ ]:
torch.cuda.empty_cache()
env = Env(R=5)

In [ ]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 512
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4
REPLAY_MEMORY_SIZE = 10000

n_actions = env.n_actions
state = env.reset()
n_observations = 1

policy_net = DeepSARSA(n_observations, n_actions).to(device)
target_net = DeepSARSA(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.Adam(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(REPLAY_MEMORY_SIZE)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        np.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[np.random.randint(0, 2)]], device=device, dtype=torch.long)

In [ ]:
def optimize_model(timestep=0, batch_num=0, avg_reward=0):
    if len(memory) < BATCH_SIZE:
        return 
    print("Optimization!")
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    #non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
    #                                      batch.next_state)), device=device, dtype=torch.bool)
    #non_final_next_states = torch.cat([s for s in batch.next_state
    #                                            if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    next_state_batch = torch.cat(batch.next_state)
    next_action_batch = torch.cat(batch.next_action)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values = target_net(next_state_batch).gather(1, next_action_batch) # SARSA update
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch # SARSA update

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # wandb.log({'loss': loss, 'timestep': timestep, 'batch': batch_num})
    wandb.log({'loss': loss, 'avg_reward': avg_reward, 'timestep': timestep}) #, 'batch': t})

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

    return loss


In [ ]:
num_episodes = 20000
num_time_per_episode = 500

wandb.config.update({
    'max_timesteps': num_episodes*num_time_per_episode,
    'batch_size': BATCH_SIZE,
    'optimizer': 'Adam',
    'learning_rate': 'default',
    'replay_memory': REPLAY_MEMORY_SIZE, # 10000
    'n_actions': n_actions,
    'n_observations': n_observations
})


In [ ]:


for i in range(num_episodes):
    prev_state = 0
    prev_action = 0
    prev_reward = 0
    for j in range(num_time_per_episode):
        cur_state = env.reset()
        cur_state = torch.tensor ([cur_state], dtype=torch.float32, device=device).unsqueeze(0)
        cur_action = select_action(cur_state)
        print(cur_action.item())
        next_state, reward = env.step(cur_action.item())

        reward = torch.tensor([reward], device=device)

        if j>0:
            memory.push(prev_state, prev_action, prev_reward, cur_state, cur_action)

        prev_state = copy.deepcopy(cur_state)
        prev_action = copy.deepcopy(cur_action)
        prev_reward = copy.deepcopy(reward)

        optimize_model(timestep=(i*num_time_per_episode) + j, avg_reward=reward.item())

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if i%100 == 0:
            SAVE_PATH = './checkpoints/DeepSARSA_{}.pt'.format(i)
            target_net.load_state_dict(policy_net.state_dict())
            torch.save(policy_net.state_dict(), SAVE_PATH)

        torch.cuda.empty_cache()


In [ ]:
transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
batch = Transition(*zip(*transitions))

# Compute a mask of non-final states and concatenate the batch elements
# (a final state would've been the one after which simulation ended)
#non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
#                                      batch.next_state)), device=device, dtype=torch.bool)
#non_final_next_states = torch.cat([s for s in batch.next_state
#                                            if s is not None])
# for i in range(len(batch.state)):
    # print(batch.state[i]
action_batch = torch.cat(batch.action)

In [ ]:
action_batch